# Test: the impact of extinction
---
Because extinction reduces the overall light, if it is not taken into account during the fit, then the SFH estimated by ppxf (particularly the stellar mass) may be inaccurate.
* Test 1: extincted vs. non-extincted WITHOUT mpoly in ppxf: how does it fare?
* Test 2: extincted vs. non-extincted WITH mpoly in ppxf: what does the best-fit polynomial look like? How accurate is the recovered SFH?


In [5]:
%matplotlib widget

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [20]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot
from ppxftests.plot_ppxf_summary import plot_ppxf_summary

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


## How does the reddening curve change as a function of $A_V$?
---
If the reddening curve takes on a relatively unique shape for each $A_V$, then it might be possible to determine the $A_V$ from the polynomial fit. We could then re-scale the SFH to recover accurate masses.

In [44]:
lambda_vals_A = np.linspace(3500, 9000, 4500)
lambda_0_A = 6000
lambda_0_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_0_A))
A_V_vals = np.linspace(0, 10, 11)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
cmap_A_V = matplotlib.cm.get_cmap("magma", len(A_V_vals))
for aa, A_V in enumerate(A_V_vals):
    A_vals = extinction.fm07(lambda_vals_A, a_v=A_V, unit="aa")
    ext_curve = 10**(-0.4 * A_vals)
    ax.plot(lambda_vals_A, ext_curve / ext_curve[lambda_0_idx], color=cmap_A_V(aa), label=r"$A_V = %.2f$" % A_V)
ax.set_xlabel("Wavelength (Å)")
ax.set_ylabel(r"$10^{-0.4 A(\lambda)} / 10^{-0.4 A(\lambda_0)}$")
ax.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## What does a heavily-extincted AGN continuum look like?
---

In [45]:
lambda_vals_A = np.linspace(3500, 9000, 4500)
lambda_0_A = 4020
lambda_0_idx = np.nanargmin(np.abs(lambda_vals_A - lambda_0_A))

alpha_nu = 1.0
alpha_lambda = 2 - alpha_nu
x_AGN = 1.0
F_0 = x_AGN / lambda_0_A**(-alpha_lambda)
F_agn = F_0 * lambda_vals_A**(-alpha_lambda)

A_V_vals = np.linspace(0, 10, 11)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))
cmap_A_V = matplotlib.cm.get_cmap("magma", len(A_V_vals))
for aa, A_V in enumerate(A_V_vals):
    A_vals = extinction.fm07(lambda_vals_A, a_v=A_V, unit="aa")
    ext_curve = 10**(-0.4 * A_vals)
    ax.plot(lambda_vals_A, F_agn * ext_curve, color=cmap_A_V(aa), label=r"$A_V = %.2f$" % A_V)
ax.set_xlabel("Wavelength (Å)")
ax.set_ylabel(r"$F_{\rm AGN} \times 10^{-0.4 A(\lambda)}$")
ax.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Run `ppxf` on spectra with extinction applied
---

In [8]:
###############################################################################
# Load a realistic SFH
###############################################################################
isochrones = "Padova"
SNR = 100
z = 0
niters = 20
nthreads = 20

x_AGN_input=0.5
alpha_nu_input=1.0

gal = 10
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)
M_tot = np.nansum(sfh_mw_input)

###############################################################################
# Create spectrum
###############################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

spec_agn, spec_agn_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=True, x_AGN=x_AGN_input, alpha_nu=alpha_nu_input,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=False)

# Apply extinction
A_V = 1.0
A_vals = extinction.fm07(lambda_vals_A, a_v=A_V, unit="aa")
ext_factor = 10**(-0.4 * A_vals)

spec_ext = spec * ext_factor
spec_ext_err = spec_err * ext_factor
spec_agn_ext = spec_agn * ext_factor
spec_agn_ext_err = spec_agn_err * ext_factor

# Plot 
fig, axs = plt.subplots(nrows=3, figsize=(15, 7.5))
axs[0].plot(lambda_vals_A, spec, label="Stellar continuum - before applying extinction")
axs[0].plot(lambda_vals_A, spec_ext, label="Stellar continuum - after applying extinction")
axs[1].plot(lambda_vals_A, spec_agn, label="Stellar continuum with AGN - before applying extinction")
axs[1].plot(lambda_vals_A, spec_agn_ext, label="Stellar continuum with AGN - after applying extinction")
axs[2].plot(lambda_vals_A, ext_factor, label="Extinction factor")
[ax.legend() for ax in axs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
###########################################################################
# Helper function for running MC simulations
###########################################################################
def ppxf_helper(args):
    # Unpack arguments
    seed, spec, spec_err, lambda_vals_A, mdegree, reddening, fit_agn_cont = args
    
    # Add "extra" noise to the spectrum
    rng = RandomState(seed)
    noise = rng.normal(scale=spec_err)
    spec_noise = spec + noise

    # This is to mitigate the "edge effects" of the convolution with the LSF
    spec_noise[0] = -9999
    spec_noise[-1] = -9999

    # Run ppxf
    pp = run_ppxf(spec=spec_noise, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=mdegree, reddening=reddening,
                  regularisation_method="none",  
                  fit_agn_cont=fit_agn_cont,
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


In [23]:
###########################################################################
# Helper function for running our tests
###########################################################################
def test_helper(spec, spec_err, mdegree, reddening, fit_agn_cont, regul_start):
    # Input arguments
    seeds = list(np.random.randint(low=0, high=100 * niters, size=niters))
    args_list = [[s, spec, spec_err, lambda_vals_A, mdegree, reddening, fit_agn_cont] for s in seeds]

    # Run in parallel
    print(f"Running ppxf on {nthreads} threads...")
    t = time()
    with multiprocessing.Pool(nthreads) as pool:
        pp_mc_list = list(tqdm(pool.imap(ppxf_helper, args_list), total=niters))
    print(f"Total time in ppxf: {time() - t:.2f} s")

    ###########################################################################
    # Run ppxf with regularisation
    ###########################################################################
    t = time()
    pp_regul = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                        isochrones=isochrones, 
                        z=z, ngascomponents=1, mdegree=mdegree, reddening=reddening, 
                        fit_gas=False, tie_balmer=True,
                        fit_agn_cont=fit_agn_cont,
                        regularisation_method="auto",
                        delta_regul_min=1, regul_max=100e4, delta_delta_chi2_min=1, regul_start=regul_start, regul_span=50e4,
                        plotit=False, savefigs=False, interactive_mode=False)

    print(f"Total time in run_ppxf: {time() - t:.2f} seconds")

    ###########################################################################
    # Summary plots
    ###########################################################################
    plot_ppxf_summary(sfh_lw_input, sfh_mw_input, pp_regul, pp_mc_list, isochrones)

    # Check the quality of the fit
    ppxf_plot(pp_regul); plt.gcf().suptitle(f"Regularised fit; mdegree={mdegree}; reddening={reddening}; fit_agn_cont={fit_agn_cont}")
    ppxf_plot(pp_mc_list[0]); plt.gcf().suptitle(f"MC fit; mdegree={mdegree}; reddening={reddening}; fit_agn_cont={fit_agn_cont}")
    
    return pp_regul, pp_mc_list


### Spectrum: NO extinction; ppxf: NO mpoly fit

In [39]:
pp_regul, pp_mc_list = test_helper(spec=spec, spec_err=spec_err, mdegree=-1, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 46.21 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 8.21 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3885...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 27.32 s
Iteration 1: optimal regul = 50000.00; Δm = 2.89318e+11; Δregul = 2500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 2.244
----------------------------------------------------
STOPPING: Optimal regul value is >= 50000.0; using 50000.00 to produce the best fit
Total time in run_ppxf: 45.12 seconds


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<ppxf.ppxf.ppxf at 0x7fed589a9e10>,
  <ppxf.ppxf.ppxf at 0x7fed59d75cd0>])

### Spectrum: WITH extinction; ppxf: NO mpoly fit

In [15]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=-1, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 24.70 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.30 s
----------------------------------------------------
Iteration 1: Scaling noise by 9.0103...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 22.33 s
Iteration 1: optimal regul = 0.00; Δm = 0.00113064; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 16.40 s
Iteration 2: optimal regul = -1000.00; Δm = 0; Δregul = 100.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 95.079
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 12.09 s
Iteration 3: optimal regul = -1200.00; Δm = 0; Δregul = 20.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 

/home/u5708159/python/Modules/ppxftests/sfhutils.py:316: RuntimeWarning: invalid value encountered in double_scalars
  log_age_lw = np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_lw_1D[age_thresh_lower_idx:age_thresh_upper_idx])
/home/u5708159/python/Modules/ppxftests/sfhutils.py:255: RuntimeWarning: invalid value encountered in double_scalars
  log_age_mw = np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx] * np.log10(ages[age_thresh_lower_idx:age_thresh_upper_idx])) / np.nansum(sfh_mw_1D[age_thresh_lower_idx:age_thresh_upper_idx])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; NO multiplicative polynomial in fit (MC)')

### Spectrum: WITH extinction; ppxf: WITH mpoly fit

In [16]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=4, reddening=None, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 72.61 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 6.58 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3790...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 107.20 s
Iteration 1: optimal regul = 10000.00; Δm = 8.79927e+10; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 27.216
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 117.80 s
Iteration 2: optimal regul = 20000.00; Δm = 5.89922e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 18.819
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 106.11 s
Iteration 3: optimal regul = 30000.00; Δm = 3.717e+08; Δregul = 500.00 (Δregul_min

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; NO multiplicative polynomial in fit (MC)')

In [25]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mc_list[0] else None)
ax.plot(pp_regul.lam, pp_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

### Remarks 
---
* The shape of the multiplicative polynomial is pretty bang-on.
* The LW and MW age estimates are both pretty accurate.
* *But...* the *absolute value* of the polynomial is way off. This must be because the multiplicative polynomial is designed to correct for small calibration errors, and so assumes that the *total* flux is fairly accurate. From the ppxf documentation: **`MDEGREE: degree of the *multiplicative* Legendre polynomial (with mean of 1) used to correct the continuum shape during the fit (default: 0)`**
* This means that the total MASS estimate is quite off.

### Spectrum: WITH extinction; ppxf: WITH extinction keyword

In [28]:
pp_regul, pp_mc_list = test_helper(spec=spec_ext, spec_err=spec_ext_err, mdegree=-1, reddening=3.0, fit_agn_cont=False)

Running ppxf on 20 threads...



Total time in ppxf: 47.20 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 4.72 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3751...
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 46.06 s
Iteration 1: optimal regul = 10000.00; Δm = 2.55497e+11; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 44.697
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Elapsed time in PPXF (multithreaded): 102.78 s
Iteration 2: optimal regul = 20000.00; Δm = 8.26014e+08; Δregul = 500.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 33.189
----------------------------------------------------
Iteration 3: Re-running ppxf on 20 threads (iteration 3)...
Iteration 3: Elapsed time in PPXF (multithreaded): 118.42 s
Iteration 3: optimal regul = 30000.00; Δm = 1.92922e+08; Δregul = 500.00 (Δregul_mi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'WITH extinction; WITH extinction correction in fit (MC)')

In [30]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
# ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mc_list[0] else None)
ax.plot(pp_regul.lam, pp_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

### Spectrum: WITH extinction; ppxf: WITH mpoly fit; WITH AGN continuum
* In this case, the polynomial fit will apply to the WHOLE spectrum (stars + sky). In this case, the AGN continuum returned by ppxf will NOT be "extincted", and so should look like the input AGN continuum BEFORE extinction is applied.

In [21]:
p_mpoly = run_ppxf(spec=spec_agn_ext, spec_err=spec_agn_ext_err, lambda_vals_A=lambda_vals_A,
                    isochrones=isochrones, 
                    z=z, ngascomponents=1, mdegree=4, reddening=None, 
                    fit_gas=False, tie_balmer=True,
                    fit_agn_cont=True,
                    regularisation_method="auto", regul_span=50e4,
                    plotit=False, savefigs=False, interactive_mode=False)


----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.52 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3712...
Iteration 1: Regularisation parameter range: 0.0-500000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 92.51 s
Iteration 1: optimal regul = 200000.00; Δm = 1.12151e+11; Δregul = 25000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 2.741
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Regularisation parameter range: 150000.0-250000.0 (n = 21)
Iteration 2: Elapsed time in PPXF (multithreaded): 102.75 s
Iteration 2: optimal regul = 205000.00; Δm = 6.2737e+07; Δregul = 5000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.835
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.8353012901643808; using 205000.

In [22]:
p_ext = run_ppxf(spec=spec_agn_ext, spec_err=spec_agn_ext_err, lambda_vals_A=lambda_vals_A,
                    isochrones=isochrones, 
                    z=z, ngascomponents=1, mdegree=-1, reddening=1., 
                    fit_gas=False, tie_balmer=True,
                    fit_agn_cont=True,
                    regularisation_method="auto", regul_span=50e4,
                    plotit=False, savefigs=False, interactive_mode=False)


----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.78 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3700...
Iteration 1: Regularisation parameter range: 0.0-500000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 63.20 s
Iteration 1: optimal regul = 100000.00; Δm = 3.00682e+11; Δregul = 25000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 3.723
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Regularisation parameter range: 50000.0-150000.0 (n = 21)
Iteration 2: Elapsed time in PPXF (multithreaded): 67.87 s
Iteration 2: optimal regul = 105000.00; Δm = 4.71852e+08; Δregul = 5000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.343
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.34324899918360074; using 105000.

In [24]:
pp_mpoly_regul, pp_mpoly_mc_list = test_helper(spec=spec_agn_ext, spec_err=spec_agn_ext_err, mdegree=4, reddening=None, fit_agn_cont=True, regul_start=0)

Running ppxf on 20 threads...



Total time in ppxf: 70.51 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 8.75 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3580...
Iteration 1: Regularisation parameter range: 0.0-500000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 101.87 s
Iteration 1: optimal regul = 200000.00; Δm = 1.03193e+11; Δregul = 25000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 3.002
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Regularisation parameter range: 150000.0-250000.0 (n = 21)
Iteration 2: Elapsed time in PPXF (multithreaded): 113.42 s
Iteration 2: optimal regul = 210000.00; Δm = 1.23659e+08; Δregul = 5000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.767
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
# Compare the extinction curve with the polynomial from ppxf 
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_mpoly_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="mpoly (MC)" if pp == pp_mpoly_mc_list[0] else None)
ax.plot(pp_mpoly_regul.lam, pp_mpoly_regul.mpoly, color="maroon", label="mpoly (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, '$\\lambda$ (Å)')

In [31]:
# Recreate the AGN continuum fit by ppxf
# In this case, because the polynomial is applied to the WHOLE spectrum including any sky components, the best-fit "sky" component should look similar to the input
# AGN continuum BEFORE the extinction correction is applied.
lambda_norm_A = 4020
F_lambda_fit = np.zeros(len(lambda_vals_A))

fig, ax = plt.subplots(figsize=(10, 5))

# Sum up the contributions of each template from the regularised ppxf fit.
for aa, alpha_nu in enumerate(pp_mpoly_regul.alpha_nu_vals):
    alpha_lambda = 2 - alpha_nu
    w_ppxf_regul = pp_mpoly_regul.weights_agn[aa]  # Weight of this template 
    x_AGN_fit_regul = (1 / w_ppxf_regul - 1)**(-1)  # Weight converted into light fraction of stellar continuum @ 4020Å
    
    F_0 = x_AGN_fit_regul / lambda_norm_A**(-alpha_lambda)  # Compute constant
    F_lambda_aa = F_0 * lambda_vals_A**(-alpha_lambda)  # Compute this component
    F_lambda_fit += F_lambda_aa  # Add to total fitted AGN continuum
    
    # Plot each contribution.
    ax.plot(lambda_vals_A, F_lambda_aa, label=f"alpha_nu = {alpha_nu}, x_AGN = {x_AGN_fit_regul}")

# Plot the TOTAL AGN continuum from the ppxf fit.
ax.plot(lambda_vals_A, F_lambda_fit, label="Total ppxf fit", color="maroon")
# ax.plot(lambda_vals_A, F_lambda_fit / pp.mpoly, label="Total ppxf fit", color="maroon", ls="--")

# Now, evaluate the INPUT AGN continuum.
alpha_lambda_input = 2 - alpha_nu_input
F_0_input = x_AGN_input / lambda_norm_A**(-alpha_lambda_input)
F_lambda_input = F_0_input * lambda_vals_A**(-alpha_lambda_input)

# Apply the extinction correction to the INPUT AGN continuum.
# NOTE: after this, we need to re-normalise F_lambda_input_ext to be equal to x_AGN at 4020Å, 
# because both the stellar & AGN continua are affected in identical ways by the extinction. 
# i.e., F_AGN(4020Å) = x_AGN * F_*(4020Å) AFTER the extinction is applied.
F_lambda_input_ext = F_lambda_input * ext_factor  
F_lambda_input_ext *= x_AGN_input / F_lambda_input_ext[np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))]

ax.plot(lambda_vals_A, F_lambda_input, label="Input AGN continuum")
ax.plot(lambda_vals_A, F_lambda_input_ext, label="Extincted input AGN continuum")
ax.axvline(lambda_norm_A, color="k", ls="--")

ax.legend()
ax.set_xlabel("Wavelenghth (Å)")
ax.set_ylabel(r"$x_{\rm AGN}(\lambda)$")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$x_{\\rm AGN}(\\lambda)$')

### Spectrum: WITH extinction; ppxf: WITH reddening; WITH AGN continuum
* In this case, the polynomial fit will ONLY apply to the stars. In this case, the AGN continuum returned by ppxf WILL be "extincted", and so should look like the input AGN continuum AFTER it has been "extincted".

In [27]:
###############################################################
# Run ppxf 
###############################################################
pp_ext_regul, pp_ext_mc_list = test_helper(spec=spec_agn_ext, spec_err=spec_agn_ext_err, mdegree=-1, reddening=3.0, fit_agn_cont=True, regul_start=0)

Running ppxf on 20 threads...



Total time in ppxf: 54.30 s
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.00 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3751...
Iteration 1: Regularisation parameter range: 0.0-500000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 70.23 s
Iteration 1: optimal regul = 100000.00; Δm = 3.00289e+11; Δregul = 25000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 4.393
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Regularisation parameter range: 50000.0-150000.0 (n = 21)
Iteration 2: Elapsed time in PPXF (multithreaded): 74.25 s
Iteration 2: optimal regul = 105000.00; Δm = 4.69812e+08; Δregul = 5000.00 (Δregul_min = 1.00); Δχ (goal) - Δχ = 0.372
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.371

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
###############################################################
# Compare the extinction curve with the polynomial from ppxf 
###############################################################
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(lambda_vals_A, ext_factor, color="black", label=r"$10^{-0.4 A_V}$")
ax.plot(lambda_vals_A, ext_factor / np.nanmean(ext_factor), color="grey", label=r"$10^{-0.4 A_V}$ (normalised)")
for pp in pp_ext_mc_list:
    ax.plot(pp.lam, pp.mpoly, alpha=0.2, color="pink", label="$10^{-0.4 A_V}$ (MC)" if pp == pp_ext_mc_list[0] else None)
ax.plot(pp_ext_regul.lam, pp_ext_regul.mpoly, color="maroon", label="$10^{-0.4 A_V}$ (regularised)")
ax.legend()
ax.set_xlabel(r"$\lambda$ (Å)")

######################################################
# Recreate the AGN continuum fit by ppxf
######################################################
# In this case, because the extinction correction is NOT applied to the sky templates within ppxf, the best-fit "sky" spectrum should look similar to the input 
# AGN spectrum AFTER extinction has been applied.
# Compare the input AGN continuum with the fitted one
w_ppxf_regul = pp_ext_regul.weights_agn
x_AGN_fit_vals_regul = (1 / w_ppxf_regul - 1)**(-1)

lambda_norm_A = 4020
F_lambda_fit = np.zeros(len(lambda_vals_A))

fig, ax = plt.subplots(figsize=(10, 5))

# Sum up the contributions of each template from the regularised ppxf fit.
for aa, alpha_nu in enumerate(pp_ext_regul.alpha_nu_vals):
    alpha_lambda = 2 - alpha_nu
    w_ppxf_regul = pp_ext_regul.weights_agn[aa]  # Weight of this template 
    x_AGN_fit_regul = (1 / w_ppxf_regul - 1)**(-1)  # Weight converted into light fraction of stellar continuum @ 4020Å
    
    F_0 = x_AGN_fit_regul / lambda_norm_A**(-alpha_lambda)  # Compute constant
    F_lambda_aa = F_0 * lambda_vals_A**(-alpha_lambda)  # Compute this component
    F_lambda_fit += F_lambda_aa  # Add to total fitted AGN continuum
    
    # Plot each contribution.
    ax.plot(lambda_vals_A, F_lambda_aa, label=f"alpha_nu = {alpha_nu}, x_AGN = {x_AGN_fit_regul}")

# Plot the TOTAL AGN continuum from the ppxf fit.
ax.plot(lambda_vals_A, F_lambda_fit, label="Total ppxf fit")

# Now, evaluate the INPUT AGN continuum.
alpha_lambda_input = 2 - alpha_nu_input
F_0_input = x_AGN_input / lambda_norm_A**(-alpha_lambda_input)
F_lambda_input = F_0_input * lambda_vals_A**(-alpha_lambda_input)

# Apply the extinction correction to the INPUT AGN continuum.
# NOTE: after this, we need to re-normalise F_lambda_input_ext to be equal to x_AGN at 4020Å, 
# because both the stellar & AGN continua are affected in identical ways by the extinction. 
# i.e., F_AGN(4020Å) = x_AGN * F_*(4020Å) AFTER the extinction is applied.
F_lambda_input_ext = F_lambda_input * ext_factor  
F_lambda_input_ext *= x_AGN_input / F_lambda_input_ext[np.nanargmin(np.abs(lambda_vals_A - lambda_norm_A))]

ax.plot(lambda_vals_A, F_lambda_input, label="Input AGN continuum")
ax.plot(lambda_vals_A, F_lambda_input_ext, label="Extincted input AGN continuum")
ax.axvline(lambda_norm_A, color="k", ls="--")

ax.legend()
ax.set_xlabel("Wavelenghth (Å)")
ax.set_ylabel(r"$x_{\rm AGN}(\lambda)$")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$x_{\\rm AGN}(\\lambda)$')